In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import re

In [2]:
sheets_to_load = ['Germany',
                  'Italy',
                  'Poland',
                  'Switzerland',
                  'Sweden',
                  'Thailand',
                  'UK',
                  'Vietnam',
                  'Australia']

# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\SEPTEMBER\q17-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\OCTOBER\q17-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\NOVEMBER\q17-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\DECEMBER\q17-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\2025\JANUARY\q17-compilation.xlsx', sheet_name=sheets_to_load)
selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\2025\JULY\q17-compilation.xlsx', sheet_name=sheets_to_load)

Germany = selected_sheets['Germany']
Italy = selected_sheets['Italy']
Poland = selected_sheets['Poland']
Switzerland = selected_sheets['Switzerland']
Sweden = selected_sheets['Sweden']
Thailand = selected_sheets['Thailand']
UK = selected_sheets['UK']
Vietnam = selected_sheets['Vietnam']
Australia = selected_sheets['Australia']

### Germany

In [3]:
Germany.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 85 columns):
 #   Column                                                                                                                                                      Non-Null Count  Dtype  
---  ------                                                                                                                                                      --------------  -----  
 0   Response ID                                                                                                                                                 400 non-null    object 
 1   Large Roadside Posters/ billboards:BOSCH:GM -  And which of these types of outdoor advertising have you seen recently for these brands?                     24 non-null     float64
 2   On bus-stops, streetlamps or retailer carparks:BOSCH:GM -  And which of these types of outdoor advertising have you seen recently for these brands?   

In [4]:
Germany.head(2) #initial dataset

,Response ID,Large Roadside Posters/ billboards:BOSCH:GM - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:BOSCH:GM - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):BOSCH:GM - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:BOSCH:GM - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):BOSCH:GM - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):BOSCH:GM - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:BOSCH:GM - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:MIELE:GM - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:MIELE:GM - And which of these types of outdoor advertising have you seen recently for these brands?",...,At sports stadiums (e.g. on the perimeter boards):HAIER:GM - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):HAIER:GM - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:HAIER:GM - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:ROWENTA:GM - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:ROWENTA:GM - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):ROWENTA:GM - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:ROWENTA:GM - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):ROWENTA:GM - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):ROWENTA:GM - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:ROWENTA:GM - And which of these types of outdoor advertising have you seen recently for these brands?
0,GER2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GER2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,6.0,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN


In [5]:
# Unpivot the DataFrame, renaming 'Variable' to 'Offline_source'
unpivoted_GM = Germany.melt(id_vars=['Response ID'], var_name='Offline_source', value_name='Y/N')

# Create a duplicate of the 'Offline_source' column named 'Offline_source_cleaned'
unpivoted_GM['Offline_source_cleaned'] = unpivoted_GM['Offline_source']

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_GM = unpivoted_GM[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'Offline_source_cleaned' column only
unpivoted_GM['Offline_source_cleaned'] = unpivoted_GM['Offline_source_cleaned'].str.replace(
    ":GM -  And which of these types of outdoor advertising have you seen recently for these brands?", 
    "", 
    regex=False
)

print(unpivoted_GM.head())

  Response ID                                     Offline_source  \
0     GER2001  Large Roadside Posters/ billboards:BOSCH:GM - ...   
1     GER2002  Large Roadside Posters/ billboards:BOSCH:GM - ...   
2     GER2003  Large Roadside Posters/ billboards:BOSCH:GM - ...   
3     GER2004  Large Roadside Posters/ billboards:BOSCH:GM - ...   
4     GER2005  Large Roadside Posters/ billboards:BOSCH:GM - ...   

                     Offline_source_cleaned  Y/N  
0  Large Roadside Posters/ billboards:BOSCH  NaN  
1  Large Roadside Posters/ billboards:BOSCH  NaN  
2  Large Roadside Posters/ billboards:BOSCH  NaN  
3  Large Roadside Posters/ billboards:BOSCH  NaN  
4  Large Roadside Posters/ billboards:BOSCH  NaN  


In [6]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Offline_source_cleaned'
for entry in unpivoted_GM['Offline_source_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_GM['Brand'] = brand_names
unpivoted_GM['Ad_Type'] = ad_types

In [7]:
# Drop the original 'Offline_source_cleaned' column
unpivoted_GM = unpivoted_GM.drop(columns=['Offline_source_cleaned'])

# Rename 'Ad_Type' to 'Offline_source_cleaned'
unpivoted_GM = unpivoted_GM.rename(columns={'Ad_Type': 'Offline_source_cleaned'})

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_GM = unpivoted_GM[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_GM['Y/N'] = np.where(unpivoted_GM['Y/N'].isna(), 0, unpivoted_GM['Y/N'])

In [8]:
unpivoted_GM.head()

,Response ID,Offline_source,Offline_source_cleaned,Y/N,Brand
0,GER2001,Large Roadside Posters/ billboards:BOSCH:GM - ...,Large Roadside Posters/ billboards,0.0,BOSCH
1,GER2002,Large Roadside Posters/ billboards:BOSCH:GM - ...,Large Roadside Posters/ billboards,0.0,BOSCH
2,GER2003,Large Roadside Posters/ billboards:BOSCH:GM - ...,Large Roadside Posters/ billboards,0.0,BOSCH
3,GER2004,Large Roadside Posters/ billboards:BOSCH:GM - ...,Large Roadside Posters/ billboards,0.0,BOSCH
4,GER2005,Large Roadside Posters/ billboards:BOSCH:GM - ...,Large Roadside Posters/ billboards,0.0,BOSCH


In [9]:
count = unpivoted_GM['Y/N'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_GM)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     32819
1.0       191
2.0       145
3.0       120
6.0       100
5.0        99
4.0        96
99.0       30
Name: count, dtype: int64


Total number of rows: 33600


### Sweden

In [10]:
Sweden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 85 columns):
 #   Column                                                                                                                                                       Non-Null Count  Dtype  
---  ------                                                                                                                                                       --------------  -----  
 0   Response ID                                                                                                                                                  400 non-null    object 
 1   Large Roadside Posters/ billboards:ELECTROLUX:SW -  And which of these types of outdoor advertising have you seen recently for these brands?                 17 non-null     float64
 2   On bus-stops, streetlamps or retailer carparks:ELECTROLUX:SW -  And which of these types of outdoor advertising have you seen recently for these b

In [11]:
Sweden.head(2) #initial dataset

,Response ID,Large Roadside Posters/ billboards:ELECTROLUX:SW - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:ELECTROLUX:SW - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):ELECTROLUX:SW - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:ELECTROLUX:SW - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):ELECTROLUX:SW - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts Â / festivals):ELECTROLUX:SW - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:ELECTROLUX:SW - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:BOSCH:SW - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:BOSCH:SW - And which of these types of outdoor advertising have you seen recently for these brands?",...,At sports stadiums (e.g. on the perimeter boards):AEG:SW - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts Â / festivals):AEG:SW - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:AEG:SW - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:ROBOROCK:SW - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:ROBOROCK:SW - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):ROBOROCK:SW - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:ROBOROCK:SW - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):ROBOROCK:SW - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts Â / festivals):ROBOROCK:SW - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:ROBOROCK:SW - And which of these types of outdoor advertising have you seen recently for these brands?
0,SWE2001,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SWE2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Unpivot the DataFrame, renaming 'Variable' to 'Offline_source'
unpivoted_SW = Sweden.melt(id_vars=['Response ID'], var_name='Offline_source', value_name='Y/N')

# Create a duplicate of the 'Offline_source' column named 'Offline_source_cleaned'
unpivoted_SW['Offline_source_cleaned'] = unpivoted_SW['Offline_source']

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_SW = unpivoted_SW[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'Offline_source_cleaned' column only
unpivoted_SW['Offline_source_cleaned'] = unpivoted_SW['Offline_source_cleaned'].str.replace(
    ":SW -  And which of these types of outdoor advertising have you seen recently for these brands?", 
    "", 
    regex=False
)

print(unpivoted_SW.head())

  Response ID                                     Offline_source  \
0     SWE2001  Large Roadside Posters/ billboards:ELECTROLUX:...   
1     SWE2002  Large Roadside Posters/ billboards:ELECTROLUX:...   
2     SWE2003  Large Roadside Posters/ billboards:ELECTROLUX:...   
3     SWE2004  Large Roadside Posters/ billboards:ELECTROLUX:...   
4     SWE2005  Large Roadside Posters/ billboards:ELECTROLUX:...   

                          Offline_source_cleaned  Y/N  
0  Large Roadside Posters/ billboards:ELECTROLUX  NaN  
1  Large Roadside Posters/ billboards:ELECTROLUX  NaN  
2  Large Roadside Posters/ billboards:ELECTROLUX  NaN  
3  Large Roadside Posters/ billboards:ELECTROLUX  NaN  
4  Large Roadside Posters/ billboards:ELECTROLUX  NaN  


In [13]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Offline_source_cleaned'
for entry in unpivoted_SW['Offline_source_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_SW['Brand'] = brand_names
unpivoted_SW['Ad_Type'] = ad_types

In [14]:
# Drop the original 'Offline_source_cleaned' column
unpivoted_SW = unpivoted_SW.drop(columns=['Offline_source_cleaned'])

# Rename 'Ad_Type' to 'Offline_source_cleaned'
unpivoted_SW = unpivoted_SW.rename(columns={'Ad_Type': 'Offline_source_cleaned'})

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_SW = unpivoted_SW[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_SW['Y/N'] = np.where(unpivoted_SW['Y/N'].isna(), 0, unpivoted_SW['Y/N'])

In [15]:
unpivoted_SW.head()

,Response ID,Offline_source,Offline_source_cleaned,Y/N,Brand
0,SWE2001,Large Roadside Posters/ billboards:ELECTROLUX:...,Large Roadside Posters/ billboards,0.0,ELECTROLUX
1,SWE2002,Large Roadside Posters/ billboards:ELECTROLUX:...,Large Roadside Posters/ billboards,0.0,ELECTROLUX
2,SWE2003,Large Roadside Posters/ billboards:ELECTROLUX:...,Large Roadside Posters/ billboards,0.0,ELECTROLUX
3,SWE2004,Large Roadside Posters/ billboards:ELECTROLUX:...,Large Roadside Posters/ billboards,0.0,ELECTROLUX
4,SWE2005,Large Roadside Posters/ billboards:ELECTROLUX:...,Large Roadside Posters/ billboards,0.0,ELECTROLUX


In [16]:
count = unpivoted_SW['Y/N'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SW)
print(f"Total number of rows: {total_rows}")


Y/N
0.0     32925
2.0       146
1.0       122
3.0       107
5.0        94
6.0        84
4.0        76
99.0       46
Name: count, dtype: int64


Total number of rows: 33600


### Italy

In [17]:
Italy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 85 columns):
 #   Column                                                                                                                                                       Non-Null Count  Dtype  
---  ------                                                                                                                                                       --------------  -----  
 0   Response ID                                                                                                                                                  400 non-null    object 
 1   Large Roadside Posters/ billboards:SAMSUNG:IT -  And which of these types of outdoor advertising have you seen recently for these brands?                    37 non-null     float64
 2   On bus-stops, streetlamps or retailer carparks:SAMSUNG:IT -  And which of these types of outdoor advertising have you seen recently for these bran

In [18]:
Italy.head(2) #initial dataset

,Response ID,Large Roadside Posters/ billboards:SAMSUNG:IT - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:SAMSUNG:IT - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):SAMSUNG:IT - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:SAMSUNG:IT - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):SAMSUNG:IT - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):SAMSUNG:IT - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:SAMSUNG:IT - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:BOSCH:IT - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:BOSCH:IT - And which of these types of outdoor advertising have you seen recently for these brands?",...,At sports stadiums (e.g. on the perimeter boards):HAIER:IT - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):HAIER:IT - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:HAIER:IT - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:SIEMENS:IT - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:SIEMENS:IT - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):SIEMENS:IT - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:SIEMENS:IT - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):SIEMENS:IT - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):SIEMENS:IT - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:SIEMENS:IT - And which of these types of outdoor advertising have you seen recently for these brands?
0,ITA2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ITA2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Unpivot the DataFrame, renaming 'Variable' to 'Offline_source'
unpivoted_IT = Italy.melt(id_vars=['Response ID'], var_name='Offline_source', value_name='Y/N')

# Create a duplicate of the 'Offline_source' column named 'Offline_source_cleaned'
unpivoted_IT['Offline_source_cleaned'] = unpivoted_IT['Offline_source']

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_IT = unpivoted_IT[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'Offline_source_cleaned' column only
unpivoted_IT['Offline_source_cleaned'] = unpivoted_IT['Offline_source_cleaned'].str.replace(
    ":IT -  And which of these types of outdoor advertising have you seen recently for these brands?", 
    "", 
    regex=False
)

print(unpivoted_IT.head())

  Response ID                                     Offline_source  \
0     ITA2001  Large Roadside Posters/ billboards:SAMSUNG:IT ...   
1     ITA2002  Large Roadside Posters/ billboards:SAMSUNG:IT ...   
2     ITA2003  Large Roadside Posters/ billboards:SAMSUNG:IT ...   
3     ITA2004  Large Roadside Posters/ billboards:SAMSUNG:IT ...   
4     ITA2005  Large Roadside Posters/ billboards:SAMSUNG:IT ...   

                       Offline_source_cleaned  Y/N  
0  Large Roadside Posters/ billboards:SAMSUNG  NaN  
1  Large Roadside Posters/ billboards:SAMSUNG  NaN  
2  Large Roadside Posters/ billboards:SAMSUNG  NaN  
3  Large Roadside Posters/ billboards:SAMSUNG  NaN  
4  Large Roadside Posters/ billboards:SAMSUNG  NaN  


In [20]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Offline_source_cleaned'
for entry in unpivoted_IT['Offline_source_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_IT['Brand'] = brand_names
unpivoted_IT['Ad_Type'] = ad_types

In [21]:
# Drop the original 'Offline_source_cleaned' column
unpivoted_IT = unpivoted_IT.drop(columns=['Offline_source_cleaned'])

# Rename 'Ad_Type' to 'Offline_source_cleaned'
unpivoted_IT = unpivoted_IT.rename(columns={'Ad_Type': 'Offline_source_cleaned'})

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_IT = unpivoted_IT[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_IT['Y/N'] = np.where(unpivoted_IT['Y/N'].isna(), 0, unpivoted_IT['Y/N'])

In [22]:
unpivoted_IT.head()

,Response ID,Offline_source,Offline_source_cleaned,Y/N,Brand
0,ITA2001,Large Roadside Posters/ billboards:SAMSUNG:IT ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
1,ITA2002,Large Roadside Posters/ billboards:SAMSUNG:IT ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
2,ITA2003,Large Roadside Posters/ billboards:SAMSUNG:IT ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
3,ITA2004,Large Roadside Posters/ billboards:SAMSUNG:IT ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
4,ITA2005,Large Roadside Posters/ billboards:SAMSUNG:IT ...,Large Roadside Posters/ billboards,0.0,SAMSUNG


In [23]:
count = unpivoted_IT['Y/N'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_IT)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     32702
1.0       211
2.0       181
3.0       137
4.0       115
6.0       111
5.0       103
99.0       40
Name: count, dtype: int64


Total number of rows: 33600


### Australia

In [24]:
Australia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 85 columns):
 #   Column                                                                                                                                                          Non-Null Count  Dtype  
---  ------                                                                                                                                                          --------------  -----  
 0   Response ID                                                                                                                                                     400 non-null    object 
 1   Large Roadside Posters/ billboards:SAMSUNG:AU -  And which of these types of outdoor advertising have you seen recently for these brands?                       54 non-null     float64
 2   On bus-stops, streetlamps or retailer carparks:SAMSUNG:AU -  And which of these types of outdoor advertising have you seen recently fo

In [25]:
Australia.head(2) # initial dataset

,Response ID,Large Roadside Posters/ billboards:SAMSUNG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:SAMSUNG:AU - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):SAMSUNG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:SAMSUNG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):SAMSUNG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts Â / festivals):SAMSUNG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:SAMSUNG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:LG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:LG:AU - And which of these types of outdoor advertising have you seen recently for these brands?",...,At sports stadiums (e.g. on the perimeter boards):HAIER:AU - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts Â / festivals):HAIER:AU - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:HAIER:AU - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:AEG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:AEG:AU - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):AEG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:AEG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):AEG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts Â / festivals):AEG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:AEG:AU - And which of these types of outdoor advertising have you seen recently for these brands?
0,AUS2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AUS2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Unpivot the DataFrame, renaming 'Variable' to 'Offline_source'
unpivoted_AU = Australia.melt(id_vars=['Response ID'], var_name='Offline_source', value_name='Y/N')

# Create a duplicate of the 'Offline_source' column named 'Offline_source_cleaned'
unpivoted_AU['Offline_source_cleaned'] = unpivoted_AU['Offline_source']

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_AU = unpivoted_AU[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'Offline_source_cleaned' column only
unpivoted_AU['Offline_source_cleaned'] = unpivoted_AU['Offline_source_cleaned'].str.replace(
    ":AU -  And which of these types of outdoor advertising have you seen recently for these brands?", 
    "", 
    regex=False
)

print(unpivoted_AU.head())

  Response ID                                     Offline_source  \
0     AUS2001  Large Roadside Posters/ billboards:SAMSUNG:AU ...   
1     AUS2002  Large Roadside Posters/ billboards:SAMSUNG:AU ...   
2     AUS2003  Large Roadside Posters/ billboards:SAMSUNG:AU ...   
3     AUS2004  Large Roadside Posters/ billboards:SAMSUNG:AU ...   
4     AUS2005  Large Roadside Posters/ billboards:SAMSUNG:AU ...   

                       Offline_source_cleaned  Y/N  
0  Large Roadside Posters/ billboards:SAMSUNG  NaN  
1  Large Roadside Posters/ billboards:SAMSUNG  NaN  
2  Large Roadside Posters/ billboards:SAMSUNG  NaN  
3  Large Roadside Posters/ billboards:SAMSUNG  NaN  
4  Large Roadside Posters/ billboards:SAMSUNG  1.0  


In [27]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Offline_source_cleaned'
for entry in unpivoted_AU['Offline_source_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_AU['Brand'] = brand_names
unpivoted_AU['Ad_Type'] = ad_types

In [28]:
# Drop the original 'Offline_source_cleaned' column
unpivoted_AU = unpivoted_AU.drop(columns=['Offline_source_cleaned'])

# Rename 'Ad_Type' to 'Offline_source_cleaned'
unpivoted_AU = unpivoted_AU.rename(columns={'Ad_Type': 'Offline_source_cleaned'})

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_AU = unpivoted_AU[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_AU['Y/N'] = np.where(unpivoted_AU['Y/N'].isna(), 0, unpivoted_AU['Y/N'])

In [29]:
unpivoted_AU.head()

,Response ID,Offline_source,Offline_source_cleaned,Y/N,Brand
0,AUS2001,Large Roadside Posters/ billboards:SAMSUNG:AU ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
1,AUS2002,Large Roadside Posters/ billboards:SAMSUNG:AU ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
2,AUS2003,Large Roadside Posters/ billboards:SAMSUNG:AU ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
3,AUS2004,Large Roadside Posters/ billboards:SAMSUNG:AU ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
4,AUS2005,Large Roadside Posters/ billboards:SAMSUNG:AU ...,Large Roadside Posters/ billboards,1.0,SAMSUNG


In [30]:
count = unpivoted_AU['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_AU)
print(f"Total number of rows: {total_rows}")

Brand
SAMSUNG         2800
LG              2800
DYSON           2800
WESTINGHOUSE    2800
BOSCH           2800
HISENSE         2800
MIELE           2800
SMEG            2800
ELECTROLUX      2800
HAIER           2800
AEG             2800
Name: count, dtype: int64


Total number of rows: 33600


In [31]:
# Replace NaN values in 'Brand' column with 'V Zug'
unpivoted_AU['Brand'] = unpivoted_AU['Brand'].fillna('FISHER&PAYKEL')

# Check the count of NaN values after replacement
na_count = unpivoted_AU['Brand'].isna().sum()
print("Number of NaN values in Brand after replacement:", na_count)

Number of NaN values in Brand after replacement: 0


In [32]:
na_count = unpivoted_AU['Brand'].isna().sum()
print("Number of NaN values in Brand:", na_count)

Number of NaN values in Brand: 0


### UK

In [33]:
UK.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 85 columns):
 #   Column                                                                                                                                                     Non-Null Count  Dtype  
---  ------                                                                                                                                                     --------------  -----  
 0   Response ID                                                                                                                                                400 non-null    object 
 1   Large Roadside Posters/ billboards:SAMSUNG:UK -  And which of these types of outdoor advertising have you seen recently for these brands?                  21 non-null     float64
 2   On bus-stops, streetlamps or retailer carparks:SAMSUNG:UK -  And which of these types of outdoor advertising have you seen recently for these brands?     

In [34]:
UK.head(2) # initial dataset

,Response ID,Large Roadside Posters/ billboards:SAMSUNG:UK - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:SAMSUNG:UK - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):SAMSUNG:UK - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:SAMSUNG:UK - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):SAMSUNG:UK - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts Â / festivals):SAMSUNG:UK - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:SAMSUNG:UK - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:LG:UK - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:LG:UK - And which of these types of outdoor advertising have you seen recently for these brands?",...,At sports stadiums (e.g. on the perimeter boards):SIEMENS:UK - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts Â / festivals):SIEMENS:UK - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:SIEMENS:UK - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:NEFF:UK - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:NEFF:UK - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):NEFF:UK - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:NEFF:UK - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):NEFF:UK - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts Â / festivals):NEFF:UK - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:NEFF:UK - And which of these types of outdoor advertising have you seen recently for these brands?
0,UK2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,UK2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# Unpivot the DataFrame, renaming 'Variable' to 'Offline_source'
unpivoted_UK = UK.melt(id_vars=['Response ID'], var_name='Offline_source', value_name='Y/N')

# Create a duplicate of the 'Offline_source' column named 'Offline_source_cleaned'
unpivoted_UK['Offline_source_cleaned'] = unpivoted_UK['Offline_source']

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_UK = unpivoted_UK[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'Offline_source_cleaned' column only
unpivoted_UK['Offline_source_cleaned'] = unpivoted_UK['Offline_source_cleaned'].str.replace(
    ":UK -  And which of these types of outdoor advertising have you seen recently for these brands?", 
    "", 
    regex=False
)

print(unpivoted_UK.head())

  Response ID                                     Offline_source  \
0      UK2001  Large Roadside Posters/ billboards:SAMSUNG:UK ...   
1      UK2002  Large Roadside Posters/ billboards:SAMSUNG:UK ...   
2      UK2003  Large Roadside Posters/ billboards:SAMSUNG:UK ...   
3      UK2004  Large Roadside Posters/ billboards:SAMSUNG:UK ...   
4      UK2005  Large Roadside Posters/ billboards:SAMSUNG:UK ...   

                       Offline_source_cleaned  Y/N  
0  Large Roadside Posters/ billboards:SAMSUNG  NaN  
1  Large Roadside Posters/ billboards:SAMSUNG  NaN  
2  Large Roadside Posters/ billboards:SAMSUNG  NaN  
3  Large Roadside Posters/ billboards:SAMSUNG  NaN  
4  Large Roadside Posters/ billboards:SAMSUNG  NaN  


In [36]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Offline_source_cleaned'
for entry in unpivoted_UK['Offline_source_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_UK['Brand'] = brand_names
unpivoted_UK['Ad_Type'] = ad_types

In [37]:
# Drop the original 'Offline_source_cleaned' column
unpivoted_UK = unpivoted_UK.drop(columns=['Offline_source_cleaned'])

# Rename 'Ad_Type' to 'Offline_source_cleaned'
unpivoted_UK = unpivoted_UK.rename(columns={'Ad_Type': 'Offline_source_cleaned'})

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_UK = unpivoted_UK[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_UK['Y/N'] = np.where(unpivoted_UK['Y/N'].isna(), 0, unpivoted_UK['Y/N'])

In [38]:
unpivoted_UK.head()

,Response ID,Offline_source,Offline_source_cleaned,Y/N,Brand
0,UK2001,Large Roadside Posters/ billboards:SAMSUNG:UK ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
1,UK2002,Large Roadside Posters/ billboards:SAMSUNG:UK ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
2,UK2003,Large Roadside Posters/ billboards:SAMSUNG:UK ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
3,UK2004,Large Roadside Posters/ billboards:SAMSUNG:UK ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
4,UK2005,Large Roadside Posters/ billboards:SAMSUNG:UK ...,Large Roadside Posters/ billboards,0.0,SAMSUNG


In [39]:
count = unpivoted_UK['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_UK)
print(f"Total number of rows: {total_rows}")

Brand
SAMSUNG     2800
LG          2800
HOTPOINT    2800
BOSCH       2800
DYSON       2800
BEKO        2800
SHARK       2800
ZANUSSI     2800
MIELE       2800
AEG         2800
SIEMENS     2800
NEFF        2800
Name: count, dtype: int64


Total number of rows: 33600


### Poland

In [40]:
Poland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 85 columns):
 #   Column                                                                                                                                                       Non-Null Count  Dtype  
---  ------                                                                                                                                                       --------------  -----  
 0   Response ID                                                                                                                                                  400 non-null    object 
 1   Large Roadside Posters/ billboards:SAMSUNG:PO -  And which of these types of outdoor advertising have you seen recently for these brands?                    22 non-null     float64
 2   On bus-stops, streetlamps or retailer carparks:SAMSUNG:PO -  And which of these types of outdoor advertising have you seen recently for these bran

In [41]:
Poland.head(2) # initial dataset

,Response ID,Large Roadside Posters/ billboards:SAMSUNG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:SAMSUNG:PO - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):SAMSUNG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:SAMSUNG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):SAMSUNG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):SAMSUNG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:SAMSUNG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:BOSCH:PO - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:BOSCH:PO - And which of these types of outdoor advertising have you seen recently for these brands?",...,At sports stadiums (e.g. on the perimeter boards):MIELE:PO - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):MIELE:PO - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:MIELE:PO - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:AEG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:AEG:PO - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):AEG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:AEG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):AEG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):AEG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:AEG:PO - And which of these types of outdoor advertising have you seen recently for these brands?
0,POL2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,POL2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# Unpivot the DataFrame, renaming 'Variable' to 'Offline_source'
unpivoted_PO = Poland.melt(id_vars=['Response ID'], var_name='Offline_source', value_name='Y/N')

# Create a duplicate of the 'Offline_source' column named 'Offline_source_cleaned'
unpivoted_PO['Offline_source_cleaned'] = unpivoted_PO['Offline_source']

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_PO = unpivoted_PO[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'Offline_source_cleaned' column only
unpivoted_PO['Offline_source_cleaned'] = unpivoted_PO['Offline_source_cleaned'].str.replace(
    ":PO -  And which of these types of outdoor advertising have you seen recently for these brands?", 
    "", 
    regex=False
)

print(unpivoted_PO.head())

  Response ID                                     Offline_source  \
0     POL2001  Large Roadside Posters/ billboards:SAMSUNG:PO ...   
1     POL2002  Large Roadside Posters/ billboards:SAMSUNG:PO ...   
2     POL2003  Large Roadside Posters/ billboards:SAMSUNG:PO ...   
3     POL2004  Large Roadside Posters/ billboards:SAMSUNG:PO ...   
4     POL2005  Large Roadside Posters/ billboards:SAMSUNG:PO ...   

                       Offline_source_cleaned  Y/N  
0  Large Roadside Posters/ billboards:SAMSUNG  NaN  
1  Large Roadside Posters/ billboards:SAMSUNG  NaN  
2  Large Roadside Posters/ billboards:SAMSUNG  NaN  
3  Large Roadside Posters/ billboards:SAMSUNG  NaN  
4  Large Roadside Posters/ billboards:SAMSUNG  NaN  


In [43]:
# import re

# # Initialize lists to store processed data
# brand_names = []
# ad_types = []

# # Process each item in 'Offline_source_cleaned'
# for entry in unpivoted_PO['Offline_source_cleaned']:
#     # Ensure entry is a string, replace NaN/float values with empty string
#     entry = str(entry) if isinstance(entry, str) else ""

#     # Extract brand name (e.g., Samsung, LG, etc.)
#     brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
#     if brand:
#         brand_name = brand.group(1)
#     else:
#         brand_name = None
    
#     # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
#     ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
#     if ad_type:
#         ad_type_name = ad_type.group(0).strip()
#     else:
#         ad_type_name = None

#     # Append to lists
#     brand_names.append(brand_name)
#     ad_types.append(ad_type_name)

# # Add extracted information to the DataFrame
# unpivoted_PO['Brand'] = brand_names
# unpivoted_PO['Ad_Type'] = ad_types


In [44]:
# import re

# # Initialize lists to store processed data
# brand_names = []
# ad_types = []

# # Process each item in 'Offline_source_cleaned'
# for entry in unpivoted_PO['Offline_source_cleaned']:
#     # Convert entry to a string to avoid TypeError
#     entry_str = str(entry)
    
#     # Extract brand name (e.g., Samsung, LG, etc.)
#     brand = re.search(r":([A-Za-z]+)$", entry_str)  # Adjusted to capture brand names at the end of the string
#     if brand:
#         brand_name = brand.group(1)
#     else:
#         brand_name = None
    
#     # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
#     ad_type = re.search(r"^[^:]+", entry_str)  # Capture everything before the first colon
#     if ad_type:
#         ad_type_name = ad_type.group(0).strip()
#     else:
#         ad_type_name = None

#     # Append to lists
#     brand_names.append(brand_name)
#     ad_types.append(ad_type_name)

# # Add extracted information to the DataFrame
# unpivoted_PO['Brand'] = brand_names
# unpivoted_PO['Ad_Type'] = ad_types


In [45]:
import re

# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Offline_source_cleaned'
for entry in unpivoted_PO['Offline_source_cleaned']:
    # Convert entry to a string to avoid TypeError
    entry_str = str(entry)
    
    # Debugging print to check the entry format
    print("Processing entry:", entry_str)
    
    # Extract brand name (e.g., Samsung, LG, etc.)
    # Adjust regex if brand names are separated by spaces or other characters
    brand = re.search(r":\s*([A-Za-z]+)$", entry_str)  # Allows optional whitespace after the colon
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry_str)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_PO['Brand'] = brand_names
unpivoted_PO['Ad_Type'] = ad_types

# Optional: Check the resulting DataFrame
print(unpivoted_PO[['Offline_source_cleaned', 'Brand', 'Ad_Type']])


Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large 

In [46]:
# # Initialize lists to store processed data
# brand_names = []
# ad_types = []

# # Process each item in 'Offline_source_cleaned'
# for entry in unpivoted_PO['Offline_source_cleaned']:
#     # Extract brand name (e.g., Samsung, LG, etc.)
#     brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
#     if brand:
#         brand_name = brand.group(1)
#     else:
#         brand_name = None
    
#     # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
#     ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
#     if ad_type:
#         ad_type_name = ad_type.group(0).strip()
#     else:
#         ad_type_name = None

#     # Append to lists
#     brand_names.append(brand_name)
#     ad_types.append(ad_type_name)

# # Add extracted information to the DataFrame
# unpivoted_PO['Brand'] = brand_names
# unpivoted_PO['Ad_Type'] = ad_types

In [47]:
# Drop the original 'Offline_source_cleaned' column
unpivoted_PO = unpivoted_PO.drop(columns=['Offline_source_cleaned'])

# Rename 'Ad_Type' to 'Offline_source_cleaned'
unpivoted_PO = unpivoted_PO.rename(columns={'Ad_Type': 'Offline_source_cleaned'})

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_PO = unpivoted_PO[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_PO['Y/N'] = np.where(unpivoted_PO['Y/N'].isna(), 0, unpivoted_PO['Y/N'])

In [48]:
count = unpivoted_PO['Y/N'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_PO)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     32993
1.0       135
2.0       121
3.0       104
6.0        77
4.0        74
5.0        73
99.0       23
Name: count, dtype: int64


Total number of rows: 33600


In [49]:
unpivoted_PO.head()

,Response ID,Offline_source,Offline_source_cleaned,Y/N,Brand
0,POL2001,Large Roadside Posters/ billboards:SAMSUNG:PO ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
1,POL2002,Large Roadside Posters/ billboards:SAMSUNG:PO ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
2,POL2003,Large Roadside Posters/ billboards:SAMSUNG:PO ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
3,POL2004,Large Roadside Posters/ billboards:SAMSUNG:PO ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
4,POL2005,Large Roadside Posters/ billboards:SAMSUNG:PO ...,Large Roadside Posters/ billboards,0.0,SAMSUNG


### Vietnam

In [50]:
Vietnam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 85 columns):
 #   Column                                                                                                                                                       Non-Null Count  Dtype  
---  ------                                                                                                                                                       --------------  -----  
 0   Response ID                                                                                                                                                  400 non-null    object 
 1   Large Roadside Posters/ billboards:SAMSUNG:VN -  And which of these types of outdoor advertising have you seen recently for these brands?                    79 non-null     float64
 2   On bus-stops, streetlamps or retailer carparks:SAMSUNG:VN -  And which of these types of outdoor advertising have you seen recently for these bran

In [51]:
Vietnam.head(2) # initial dataset

,Response ID,Large Roadside Posters/ billboards:SAMSUNG:VN - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:SAMSUNG:VN - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):SAMSUNG:VN - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:SAMSUNG:VN - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):SAMSUNG:VN - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):SAMSUNG:VN - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:SAMSUNG:VN - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:PANASONIC:VN - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:PANASONIC:VN - And which of these types of outdoor advertising have you seen recently for these brands?",...,At sports stadiums (e.g. on the perimeter boards):BOSCH:VN - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):BOSCH:VN - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:BOSCH:VN - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:MIDEA:VN - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:MIDEA:VN - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):MIDEA:VN - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:MIDEA:VN - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):MIDEA:VN - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):MIDEA:VN - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:MIDEA:VN - And which of these types of outdoor advertising have you seen recently for these brands?
0,VN02001,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VN02002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
# Unpivot the DataFrame, renaming 'Variable' to 'Offline_source'
unpivoted_VN = Vietnam.melt(id_vars=['Response ID'], var_name='Offline_source', value_name='Y/N')

# Create a duplicate of the 'Offline_source' column named 'Offline_source_cleaned'
unpivoted_VN['Offline_source_cleaned'] = unpivoted_VN['Offline_source']

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_VN = unpivoted_VN[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'Offline_source_cleaned' column only
unpivoted_VN['Offline_source_cleaned'] = unpivoted_VN['Offline_source_cleaned'].str.replace(
    ":VN -  And which of these types of outdoor advertising have you seen recently for these brands?", 
    "", 
    regex=False
)

print(unpivoted_VN.head())

  Response ID                                     Offline_source  \
0     VN02001  Large Roadside Posters/ billboards:SAMSUNG:VN ...   
1     VN02002  Large Roadside Posters/ billboards:SAMSUNG:VN ...   
2     VN02003  Large Roadside Posters/ billboards:SAMSUNG:VN ...   
3     VN02004  Large Roadside Posters/ billboards:SAMSUNG:VN ...   
4     VN02005  Large Roadside Posters/ billboards:SAMSUNG:VN ...   

                       Offline_source_cleaned  Y/N  
0  Large Roadside Posters/ billboards:SAMSUNG  1.0  
1  Large Roadside Posters/ billboards:SAMSUNG  NaN  
2  Large Roadside Posters/ billboards:SAMSUNG  NaN  
3  Large Roadside Posters/ billboards:SAMSUNG  NaN  
4  Large Roadside Posters/ billboards:SAMSUNG  NaN  


In [53]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Offline_source_cleaned'
for entry in unpivoted_VN['Offline_source_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_VN['Brand'] = brand_names
unpivoted_VN['Ad_Type'] = ad_types

In [54]:
# Drop the original 'Offline_source_cleaned' column
unpivoted_VN = unpivoted_VN.drop(columns=['Offline_source_cleaned'])

# Rename 'Ad_Type' to 'Offline_source_cleaned'
unpivoted_VN = unpivoted_VN.rename(columns={'Ad_Type': 'Offline_source_cleaned'})

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_VN = unpivoted_VN[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_VN['Y/N'] = np.where(unpivoted_VN['Y/N'].isna(), 0, unpivoted_VN['Y/N'])

In [55]:
unpivoted_VN.head()

,Response ID,Offline_source,Offline_source_cleaned,Y/N,Brand
0,VN02001,Large Roadside Posters/ billboards:SAMSUNG:VN ...,Large Roadside Posters/ billboards,1.0,SAMSUNG
1,VN02002,Large Roadside Posters/ billboards:SAMSUNG:VN ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
2,VN02003,Large Roadside Posters/ billboards:SAMSUNG:VN ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
3,VN02004,Large Roadside Posters/ billboards:SAMSUNG:VN ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
4,VN02005,Large Roadside Posters/ billboards:SAMSUNG:VN ...,Large Roadside Posters/ billboards,0.0,SAMSUNG


In [56]:
count = unpivoted_VN['Y/N'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_VN)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     30666
1.0       572
3.0       529
2.0       525
5.0       471
6.0       425
4.0       382
99.0       30
Name: count, dtype: int64


Total number of rows: 33600


### Switzerland

In [57]:
Switzerland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 85 columns):
 #   Column                                                                                                                                                       Non-Null Count  Dtype  
---  ------                                                                                                                                                       --------------  -----  
 0   Response ID                                                                                                                                                  400 non-null    object 
 1   Large Roadside Posters/ billboards:V ZUG:SZ -  And which of these types of outdoor advertising have you seen recently for these brands?                      14 non-null     float64
 2   On bus-stops, streetlamps or retailer carparks:V ZUG:SZ -  And which of these types of outdoor advertising have you seen recently for these brands

In [58]:
Switzerland.head(2) # initial dataset

,Response ID,Large Roadside Posters/ billboards:V ZUG:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:V ZUG:SZ - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):V ZUG:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:V ZUG:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):V ZUG:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):V ZUG:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:V ZUG:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:ELECTROLUX:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:ELECTROLUX:SZ - And which of these types of outdoor advertising have you seen recently for these brands?",...,At sports stadiums (e.g. on the perimeter boards):LIEBHERR:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):LIEBHERR:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:LIEBHERR:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:SCHULTHESS:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:SCHULTHESS:SZ - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):SCHULTHESS:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:SCHULTHESS:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):SCHULTHESS:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):SCHULTHESS:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:SCHULTHESS:SZ - And which of these types of outdoor advertising have you seen recently for these brands?
0,SWIT2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,99.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SWIT2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
# Unpivot the DataFrame, renaming 'Variable' to 'Offline_source'
unpivoted_SZ = Switzerland.melt(id_vars=['Response ID'], var_name='Offline_source', value_name='Y/N')

# Create a duplicate of the 'Offline_source' column named 'Offline_source_cleaned'
unpivoted_SZ['Offline_source_cleaned'] = unpivoted_SZ['Offline_source']

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_SZ = unpivoted_SZ[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'Offline_source_cleaned' column only
unpivoted_SZ['Offline_source_cleaned'] = unpivoted_SZ['Offline_source_cleaned'].str.replace(
    ":SZ -  And which of these types of outdoor advertising have you seen recently for these brands?", 
    "", 
    regex=False
)

print(unpivoted_SZ.head())

  Response ID                                     Offline_source  \
0    SWIT2001  Large Roadside Posters/ billboards:V ZUG:SZ - ...   
1    SWIT2002  Large Roadside Posters/ billboards:V ZUG:SZ - ...   
2    SWIT2003  Large Roadside Posters/ billboards:V ZUG:SZ - ...   
3    SWIT2004  Large Roadside Posters/ billboards:V ZUG:SZ - ...   
4    SWIT2005  Large Roadside Posters/ billboards:V ZUG:SZ - ...   

                     Offline_source_cleaned  Y/N  
0  Large Roadside Posters/ billboards:V ZUG  NaN  
1  Large Roadside Posters/ billboards:V ZUG  NaN  
2  Large Roadside Posters/ billboards:V ZUG  NaN  
3  Large Roadside Posters/ billboards:V ZUG  NaN  
4  Large Roadside Posters/ billboards:V ZUG  NaN  


In [60]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Offline_source_cleaned'
for entry in unpivoted_SZ['Offline_source_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_SZ['Brand'] = brand_names
unpivoted_SZ['Ad_Type'] = ad_types

In [61]:
# Drop the original 'Offline_source_cleaned' column
unpivoted_SZ = unpivoted_SZ.drop(columns=['Offline_source_cleaned'])

# Rename 'Ad_Type' to 'Offline_source_cleaned'
unpivoted_SZ = unpivoted_SZ.rename(columns={'Ad_Type': 'Offline_source_cleaned'})

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_SZ = unpivoted_SZ[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_SZ['Y/N'] = np.where(unpivoted_SZ['Y/N'].isna(), 0, unpivoted_SZ['Y/N'])

In [62]:
unpivoted_SZ.head()

,Response ID,Offline_source,Offline_source_cleaned,Y/N,Brand
0,SWIT2001,Large Roadside Posters/ billboards:V ZUG:SZ - ...,Large Roadside Posters/ billboards,0.0,None
1,SWIT2002,Large Roadside Posters/ billboards:V ZUG:SZ - ...,Large Roadside Posters/ billboards,0.0,None
2,SWIT2003,Large Roadside Posters/ billboards:V ZUG:SZ - ...,Large Roadside Posters/ billboards,0.0,None
3,SWIT2004,Large Roadside Posters/ billboards:V ZUG:SZ - ...,Large Roadside Posters/ billboards,0.0,None
4,SWIT2005,Large Roadside Posters/ billboards:V ZUG:SZ - ...,Large Roadside Posters/ billboards,0.0,None


In [63]:
count = unpivoted_SZ['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SZ)
print(f"Total number of rows: {total_rows}")

Brand
ELECTROLUX    2800
MIELE         2800
DYSON         2800
BOSCH         2800
SAMSUNG       2800
SIEMENS       2800
PHILIPS       2800
BAUKNECHT     2800
AEG           2800
LIEBHERR      2800
SCHULTHESS    2800
Name: count, dtype: int64


Total number of rows: 33600


In [64]:
# Replace NaN values in 'Brand' column with 'V Zug'
unpivoted_SZ['Brand'] = unpivoted_SZ['Brand'].fillna('V ZUG')

# Check the count of NaN values after replacement
na_count = unpivoted_SZ['Brand'].isna().sum()
print("Number of NaN values in Brand after replacement:", na_count)


Number of NaN values in Brand after replacement: 0


In [65]:
na_count = unpivoted_SZ['Brand'].isna().sum()
print("Number of NaN values in Brand:", na_count)

Number of NaN values in Brand: 0


### Thailand

In [66]:
Thailand.info

<bound method DataFrame.info of     Response ID  \
0        TH2001   
1        TH2002   
2        TH2003   
3        TH2004   
4        TH2005   
..          ...   
395      TH2396   
396      TH2397   
397      TH2398   
398      TH2399   
399      TH2400   

     Large Roadside Posters/ billboards:SAMSUNG:TH -  And which of these types of outdoor advertising have you seen recently for these brands?  \
0                                                  NaN                                                                                           
1                                                  NaN                                                                                           
2                                                  NaN                                                                                           
3                                                  NaN                                                                                           
4        

In [67]:
Thailand.head(2) # initial dataset

,Response ID,Large Roadside Posters/ billboards:SAMSUNG:TH - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:SAMSUNG:TH - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):SAMSUNG:TH - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:SAMSUNG:TH - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):SAMSUNG:TH - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts Â / festivals):SAMSUNG:TH - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:SAMSUNG:TH - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:LG:TH - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:LG:TH - And which of these types of outdoor advertising have you seen recently for these brands?",...,At sports stadiums (e.g. on the perimeter boards):BOSCH:TH - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts Â / festivals):BOSCH:TH - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:BOSCH:TH - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:DYSON:TH - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:DYSON:TH - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):DYSON:TH - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:DYSON:TH - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):DYSON:TH - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts Â / festivals):DYSON:TH - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:DYSON:TH - And which of these types of outdoor advertising have you seen recently for these brands?
0,TH2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TH2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
# Unpivot the DataFrame, renaming 'Variable' to 'Offline_source'
unpivoted_TH = Thailand.melt(id_vars=['Response ID'], var_name='Offline_source', value_name='Y/N')

# Create a duplicate of the 'Offline_source' column named 'Offline_source_cleaned'
unpivoted_TH['Offline_source_cleaned'] = unpivoted_TH['Offline_source']

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_TH = unpivoted_TH[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'Offline_source_cleaned' column only
unpivoted_TH['Offline_source_cleaned'] = unpivoted_TH['Offline_source_cleaned'].str.replace(
    ":TH -  And which of these types of outdoor advertising have you seen recently for these brands?", 
    "", 
    regex=False
)

print(unpivoted_TH.head())

  Response ID                                     Offline_source  \
0      TH2001  Large Roadside Posters/ billboards:SAMSUNG:TH ...   
1      TH2002  Large Roadside Posters/ billboards:SAMSUNG:TH ...   
2      TH2003  Large Roadside Posters/ billboards:SAMSUNG:TH ...   
3      TH2004  Large Roadside Posters/ billboards:SAMSUNG:TH ...   
4      TH2005  Large Roadside Posters/ billboards:SAMSUNG:TH ...   

                       Offline_source_cleaned  Y/N  
0  Large Roadside Posters/ billboards:SAMSUNG  NaN  
1  Large Roadside Posters/ billboards:SAMSUNG  NaN  
2  Large Roadside Posters/ billboards:SAMSUNG  NaN  
3  Large Roadside Posters/ billboards:SAMSUNG  NaN  
4  Large Roadside Posters/ billboards:SAMSUNG  NaN  


In [69]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Offline_source_cleaned'
for entry in unpivoted_TH['Offline_source_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_TH['Brand'] = brand_names
unpivoted_TH['Ad_Type'] = ad_types

In [70]:
# Drop the original 'Offline_source_cleaned' column
unpivoted_TH = unpivoted_TH.drop(columns=['Offline_source_cleaned'])

# Rename 'Ad_Type' to 'Offline_source_cleaned'
unpivoted_TH = unpivoted_TH.rename(columns={'Ad_Type': 'Offline_source_cleaned'})

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_TH = unpivoted_TH[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_TH['Y/N'] = np.where(unpivoted_TH['Y/N'].isna(), 0, unpivoted_TH['Y/N'])

In [71]:
unpivoted_TH.head()

,Response ID,Offline_source,Offline_source_cleaned,Y/N,Brand
0,TH2001,Large Roadside Posters/ billboards:SAMSUNG:TH ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
1,TH2002,Large Roadside Posters/ billboards:SAMSUNG:TH ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
2,TH2003,Large Roadside Posters/ billboards:SAMSUNG:TH ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
3,TH2004,Large Roadside Posters/ billboards:SAMSUNG:TH ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
4,TH2005,Large Roadside Posters/ billboards:SAMSUNG:TH ...,Large Roadside Posters/ billboards,0.0,SAMSUNG


In [72]:
count = unpivoted_TH['Y/N'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_TH)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     30622
1.0       590
2.0       539
3.0       484
5.0       441
4.0       439
6.0       431
99.0       54
Name: count, dtype: int64


Total number of rows: 33600


### Concat and save them in another csv file

In [73]:
# Concatenate the melted DataFrames
combined_q17 = pd.concat([unpivoted_GM, unpivoted_SW, unpivoted_TH, unpivoted_IT, unpivoted_PO, 
                                unpivoted_SZ, unpivoted_VN, unpivoted_AU, unpivoted_UK], ignore_index=True)

# combined_q17.dropna(subset=['Response ID'], inplace=True)

combined_q17 = combined_q17.sort_values(by='Response ID', ascending=True)

# combined_q17.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\SEPTEMBER\combined_q17.xlsx', float_format='%.0f', index=False)
# combined_q17.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\OCTOBER\combined_q17.xlsx', float_format='%.0f', index=False)
# combined_q17.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\NOVEMBER\combined_q17.xlsx', float_format='%.0f', index=False)
# combined_q17.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\DECEMBER\combined_q17.xlsx', float_format='%.0f', index=False)
# combined_q17.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\2025\JANUARY\combined_q17.xlsx', float_format='%.0f', index=False)
combined_q17.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\2025\JULY\combined_q17.xlsx', float_format='%.0f', index=False)

combined_q17.head(2)

,Response ID,Offline_source,Offline_source_cleaned,Y/N,Brand
235200,AUS2001,Large Roadside Posters/ billboards:SAMSUNG:AU ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
263600,AUS2001,"On bus-stops, streetlamps or retailer carparks...","On bus-stops, streetlamps or retailer carparks",0.0,HAIER


In [74]:
# Count blank (NaN) rows in the 'Response ID' column
blank_count = combined_q17['Response ID'].isna().sum()
print(f"Number of blank rows in 'Response ID': {blank_count}")

Number of blank rows in 'Response ID': 0


In [75]:
combined_q17.info()

<class 'pandas.core.frame.DataFrame'>
Index: 302400 entries, 235200 to 218399
Data columns (total 5 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Response ID             302400 non-null  object 
 1   Offline_source          302400 non-null  object 
 2   Offline_source_cleaned  302400 non-null  object 
 3   Y/N                     302400 non-null  float64
 4   Brand                   302400 non-null  object 
dtypes: float64(1), object(4)
memory usage: 13.8+ MB


In [76]:
count = combined_q17['Brand'].value_counts()
print(count)

Brand
SAMSUNG          25200
BOSCH            25200
DYSON            22400
LG               22400
MIELE            19600
ELECTROLUX       19600
AEG              19600
PHILIPS          16800
HAIER            14000
SIEMENS          14000
BEKO              8400
WHIRLPOOL         8400
BAUKNECHT         5600
HITACHI           5600
NEFF              5600
HOTPOINT          5600
SHARP             5600
PANASONIC         5600
HISENSE           2800
ROWENTA           2800
SMEG              2800
AMICA             2800
WESTINGHOUSE      2800
FISHER&PAYKEL     2800
ROBOROCK          2800
V ZUG             2800
LIEBHERR          2800
CYLINDA           2800
SCHULTHESS        2800
MITSUBISHI        2800
TEFAL             2800
ZANUSSI           2800
SHARK             2800
TOSHIBA           2800
XIAOMI            2800
AQUA              2800
MIDEA             2800
Name: count, dtype: int64


In [77]:
count = combined_q17['Y/N'].value_counts()
print(count)

Y/N
0.0     290105
1.0       2464
2.0       2252
3.0       2022
5.0       1809
6.0       1711
4.0       1658
99.0       379
Name: count, dtype: int64
